In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
sys.path.append('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/')

from enum import Enum

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

import tensorflow_probability as tfp
from scipy.stats import beta, truncnorm


# Local imports from model.py, data.py
from model import CovidModel, LogPoissonProb, get_logging_callbacks, Comp, Vax
from data import read_data, create_warmup
#from plots import make_all_plots

import scipy

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20}) # set plot font sizes

2021-12-05 18:31:18.985397: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-05 18:31:18.985435: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
transition_window =3

warmup_start = '20210428'
warmup_end = '20210430'
train_start = '20210501'
train_end = '20210731'
test_start = '20210801'
test_end = '20210831'

state = 'Massachusetts'
state_abbrev = 'MA'

data_dir = '../data'
covid_estim_date = '20210901'
hhs_date = '20210903'
owid_date = '20210903'

log_dir = './logs/new_warmup'


# Learning rate
learning_rate = 20

In [3]:
df = read_data(data_dir=data_dir,
               covid_estim_date=covid_estim_date,
               hhs_date=hhs_date,
               owid_date=owid_date,
               state=state, state_abbrev=state_abbrev)

In [4]:
class Comp(Enum):
    A = 0
    M = 1
    #X = 2
    #G = 3
    
class Vax(Enum):
    total = -1
    no = 0
    yes = 1

In [5]:
# get warmup arrays, splitting on vaccination status
warmup_asymp, warmup_mild, warmup_extreme = create_warmup(df, 
                                                          warmup_start, 
                                                          warmup_end,
                                                          0,0,0)

# re-combine
warmup_asymp = warmup_asymp[Vax.no.value] + warmup_asymp[Vax.yes.value]

In [6]:
synth_T_serial = 5.8
synth_rho_M = 0.76
synth_lambda_M = 4.7
synth_nu_M = 3.1

In [7]:
warmup_A_params = {}
warmup_A_params[Vax.total.value] = {}
warmup_A_params[Vax.total.value]['prior'] = []
warmup_A_params[Vax.total.value]['posterior_init'] = []

for day in range(transition_window):
    warmup_A_params[Vax.total.value]['prior'].append({'loc': warmup_asymp[day],
                                                'scale': warmup_asymp[day]/10})
    

In [8]:
x_train = tf.cast(df.loc[train_start:test_end,'Rt'].values, dtype=tf.float32)
#y_test = tf.cast(df.loc[train_start:test_end,'mild'], dtype=tf.float32)

2021-12-05 18:31:20.944998: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-05 18:31:20.945205: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-05 18:31:20.945239: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-05 18:31:20.945266: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-05 18:31:20.945289: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [9]:
A_vals = []
M_vals =[]
for day in range(transition_window):
    A_vals.append(warmup_A_params[-1]['prior'][day]['loc'])

for day in range(len(x_train)):
    yesterday_asymp = A_vals[-1]
    
    today_asymp = yesterday_asymp*x_train[day]**(1/synth_T_serial)
    A_vals.append(today_asymp)
    
    today_M = 0
    pi_M=[]
    for j in range(transition_window):
        
        
        lambda_M_fix = synth_lambda_M
        nu_M_fix = synth_nu_M
        poisson_dist = scipy.stats.poisson(lambda_M_fix)
        pi_M_j_ago = poisson_dist.logpmf(j+1)/nu_M_fix
        
        pi_M.append(pi_M_j_ago)
    
    pi_M = scipy.special.softmax(pi_M)
        
    for j in range(transition_window):
        j_ago_asymp = A_vals[day-j-1]
        today_M += j_ago_asymp*synth_rho_M*pi_M[j]
        
    M_vals.append(today_M)

In [10]:

T_serial = {}
T_serial[Vax.total.value] = {}
T_serial[Vax.total.value]['prior'] ={'loc':5.8, 'scale':1}


rho_M = {}
rho_M[Vax.total.value] = {}
rho_M[Vax.total.value]['prior'] = {'a': 31.8, 'b': 10.3}

lambda_M = {}
lambda_M[Vax.total.value] = {}
lambda_M[Vax.total.value]['prior'] = {'loc': 4.7, 'scale': 1}

nu_M = {}
nu_M[Vax.total.value] = {}
nu_M[Vax.total.value]['prior'] = {'loc': 3.1, 'scale': 1.2}




In [11]:
T_serial_scale = 1.0
rho_M_scale = 0.1
lambda_M_scale = 1.0
nu_M_scale = 1.2

warmup_scales = [0.1]*warmup_asymp

In [12]:
T_serial[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(synth_T_serial),
                                     'scale':tf.cast(tfp.math.softplus_inverse(T_serial_scale),dtype=tf.float32)}

rho_M[Vax.total.value]['posterior_init'] = {'loc': tf.cast(np.log(synth_rho_M/(1-synth_rho_M)),dtype=tf.float32),
                                      'scale':tf.cast(tfp.math.softplus_inverse(rho_M_scale),dtype=tf.float32)}

lambda_M[Vax.total.value]['posterior_init'] = {'loc': tf.cast(tfp.math.softplus_inverse(synth_lambda_M),dtype=tf.float32),
                                         'scale':tf.cast(tfp.math.softplus_inverse(lambda_M_scale),dtype=tf.float32)}

nu_M[Vax.total.value]['posterior_init'] = {'loc': tf.cast(tfp.math.softplus_inverse(synth_nu_M),dtype=tf.float32),
                                     'scale':tf.cast(tfp.math.softplus_inverse(nu_M_scale),dtype=tf.float32)}

for day in range(transition_window):
    # must be positive so reverse softplus the mean
    warmup_A_params[Vax.total.value]['posterior_init'].append({'loc': tf.cast(tfp.math.softplus_inverse(1000.0),dtype=tf.float32),
                                                         'scale': tf.cast(tfp.math.softplus_inverse(500.0),dtype=tf.float32)})#tf.cast(tfp.math.softplus_inverse(warmup_asymp[day]/10),dtype=tf.float32)})

model = CovidModel([Vax.total], [Comp.A, Comp.M],
                 transition_window,
                T_serial, rho_M, lambda_M, nu_M,
                 warmup_A_params, posterior_samples=1000, debug_disable_theta=True)

In [13]:
loss = LogPoissonProb() 
optimizer = tf.keras.optimizers.Adam(
    learning_rate=learning_rate, #momentum=0.98, nesterov=False, name='SGD'
)

In [14]:
logging_callbacks = get_logging_callbacks('/mnt/c/Users/kheut/logs/covid/bayes_learn_warmup_01')

2021-12-05 18:31:21.283000: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-05 18:31:21.283038: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-05 18:31:21.283070: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2021-12-05 18:31:21.283297: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2021-12-05 18:31:21.283352: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory
2021-12-05 18:31:21.283365: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1666] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be 

In [15]:
model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
model.fit(x=np.asarray([x_train]), y=np.asarray([M_vals]),
         epochs=1000, batch_size=0,
        callbacks=logging_callbacks)

2021-12-05 18:31:21.348833: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 51865.0938


2021-12-05 18:31:23.437559: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2021-12-05 18:31:23.540506: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-05 18:31:23.540537: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-05 18:31:23.540736: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1666] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.


Epoch 2/1000
1/1 [==============================] - ETA: 0s - loss: 50976.4883

2021-12-05 18:31:25.853393: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-05 18:31:25.857127: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1757] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2021-12-05 18:31:25.981339: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-12-05 18:31:26.294363: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-05 18:31:27.165594: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /mnt/c/Users/kheut/logs/covid/bayes_learn_warmup_01/train/plugins/profile/2021_12_05_18_31_26

2021-12-05 18:31:27.888609: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to /mnt/c/Users/kheut/logs/covid/bayes_learn_warmup_01/train/plugins/profil

...

1/1 [==============================] - 2s 2s/step - loss: 1748.3004
Epoch 593/1000
1/1 [==============================] - 2s 2s/step - loss: 1788.0347
Epoch 594/1000
1/1 [==============================] - 2s 2s/step - loss: 1720.6025
Epoch 595/1000
1/1 [==============================] - 2s 2s/step - loss: 1819.7522
Epoch 596/1000
1/1 [==============================] - 2s 2s/step - loss: 1742.0668
Epoch 597/1000
1/1 [==============================] - 2s 2s/step - loss: 1726.1243
Epoch 598/1000
1/1 [==============================] - 2s 2s/step - loss: 1853.9061
Epoch 599/1000
1/1 [==============================] - 2s 2s/step - loss: 1822.1631
Epoch 600/1000
1/1 [==============================] - 2s 2s/step - loss: 1713.3281
Epoch 601/1000
1/1 [==============================] - 2s 2s/step - loss: 1793.5963
Epoch 602/1000
1/1 [==============================] - 2s 2s/step - loss: 1866.4672
Epoch 603/1000
1/1 [==============================] - 2s 2s/step - loss: 1824.3647
Epoch 604/1000
1/1 

1/1 [==============================] - 2s 2s/step - loss: 1830.4789
Epoch 691/1000
1/1 [==============================] - 2s 2s/step - loss: 1773.2559
Epoch 692/1000
1/1 [==============================] - 2s 2s/step - loss: 1761.4639
Epoch 693/1000
1/1 [==============================] - 2s 2s/step - loss: 1835.7097
Epoch 694/1000
1/1 [==============================] - 2s 2s/step - loss: 1745.8038
Epoch 695/1000
1/1 [==============================] - 2s 2s/step - loss: 1714.0636
Epoch 696/1000
1/1 [==============================] - 2s 2s/step - loss: 1779.0756
Epoch 697/1000
1/1 [==============================] - 2s 2s/step - loss: 1819.1709
Epoch 698/1000
1/1 [==============================] - 2s 2s/step - loss: 1844.8047
Epoch 699/1000
1/1 [==============================] - 2s 2s/step - loss: 1750.2462
Epoch 700/1000
1/1 [==============================] - 2s 2s/step - loss: 1658.8790
Epoch 701/1000
1/1 [==============================] - 2s 2s/step - loss: 1766.7511
Epoch 702/1000
1/1 

1/1 [==============================] - 2s 2s/step - loss: 1804.6499
Epoch 789/1000
1/1 [==============================] - 2s 2s/step - loss: 1842.2799
Epoch 790/1000
1/1 [==============================] - 2s 2s/step - loss: 1779.4421
Epoch 791/1000
1/1 [==============================] - 2s 2s/step - loss: 1866.5339
Epoch 792/1000
1/1 [==============================] - 2s 2s/step - loss: 1727.9998
Epoch 793/1000
1/1 [==============================] - 2s 2s/step - loss: 1815.5421
Epoch 794/1000
1/1 [==============================] - 2s 2s/step - loss: 1780.9999
Epoch 795/1000
1/1 [==============================] - 2s 2s/step - loss: 1821.6072
Epoch 796/1000
1/1 [==============================] - 2s 2s/step - loss: 1833.3037
Epoch 797/1000
1/1 [==============================] - 2s 2s/step - loss: 1869.9686
Epoch 798/1000
1/1 [==============================] - 2s 2s/step - loss: 1783.4591
Epoch 799/1000
1/1 [==============================] - 2s 2s/step - loss: 1697.1545
Epoch 800/1000
1/1 

1/1 [==============================] - 2s 2s/step - loss: 1919.0458
Epoch 887/1000
1/1 [==============================] - 2s 2s/step - loss: 1916.6235
Epoch 888/1000
1/1 [==============================] - 2s 2s/step - loss: 1993.5557
Epoch 889/1000
1/1 [==============================] - 2s 2s/step - loss: 1818.2864
Epoch 890/1000
1/1 [==============================] - 2s 2s/step - loss: 1799.9124
Epoch 891/1000
1/1 [==============================] - 2s 2s/step - loss: 1739.1924
Epoch 892/1000
1/1 [==============================] - 2s 2s/step - loss: 1805.4735
Epoch 893/1000
1/1 [==============================] - 2s 2s/step - loss: 1931.7731
Epoch 894/1000
1/1 [==============================] - 2s 2s/step - loss: 1791.9869
Epoch 895/1000
1/1 [==============================] - 2s 2s/step - loss: 1725.5192
Epoch 896/1000
1/1 [==============================] - 2s 2s/step - loss: 1775.2277
Epoch 897/1000
1/1 [==============================] - 2s 2s/step - loss: 1939.4608
Epoch 898/1000
1/1 

1/1 [==============================] - 2s 2s/step - loss: 1832.6398
Epoch 985/1000
1/1 [==============================] - 2s 2s/step - loss: 1799.8389
Epoch 986/1000
1/1 [==============================] - 2s 2s/step - loss: 1761.6560
Epoch 987/1000
1/1 [==============================] - 2s 2s/step - loss: 1785.4032
Epoch 988/1000
1/1 [==============================] - 2s 2s/step - loss: 1786.7524
Epoch 989/1000
1/1 [==============================] - 2s 2s/step - loss: 1776.9867
Epoch 990/1000
1/1 [==============================] - 2s 2s/step - loss: 1744.0845
Epoch 991/1000
1/1 [==============================] - 2s 2s/step - loss: 1823.5909
Epoch 992/1000
1/1 [==============================] - 2s 2s/step - loss: 1845.6556
Epoch 993/1000
1/1 [==============================] - 2s 2s/step - loss: 1710.8326
Epoch 994/1000
1/1 [==============================] - 2s 2s/step - loss: 1774.6180
Epoch 995/1000
1/1 [==============================] - 2s 2s/step - loss: 1774.2706
Epoch 996/1000
1/1 

In [16]:
model.trainable_variables

[<tf.Variable 'warmup_A_loc_0_-1:0' shape=() dtype=float32, numpy=2507.1702>,
 <tf.Variable 'warmup_A_scale_0_-1:0' shape=() dtype=float32, numpy=123.81848>,
 <tf.Variable 'warmup_A_loc_1_-1:0' shape=() dtype=float32, numpy=2597.5881>,
 <tf.Variable 'warmup_A_scale_1_-1:0' shape=() dtype=float32, numpy=103.140945>,
 <tf.Variable 'warmup_A_loc_2_-1:0' shape=() dtype=float32, numpy=2437.437>,
 <tf.Variable 'warmup_A_scale_2_-1:0' shape=() dtype=float32, numpy=5.1279593>]

In [18]:
warmup_asymp

array([2518.60923786, 2520.22371777, 2518.626799  ])